In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import sys
sys.path.append("../../")
import CCG.utils
import re

In [31]:
Selection = ['NER']


def count_sublist(lis,sublis):
    cnt = 0
    if len(sublis)>len(lis):
        return cnt
    len_sub = len(sublis)
    for st in range(len(lis)-len(sublis)+1):
        if lis[st:st+len_sub]==sublis:
            cnt+=1
    return cnt


#function for $And
def merge(x,y):
    if type(x)!= tuple:
        x = [x]
    if type(y)!= tuple:
        y = [y]
    x = list(x)
    y = list(y)
    return tuple(x+y)

#function for $Is
def IsFunc(ws,ps,c):
    if isinstance(ps,tuple):
        bool_list  = []
        for p in ps:
            if isinstance(ws, tuple):
                if ws[0] in Selection:
                    bool_list.append(p(ws)(c))
                else:
                    bool_list.append(all([p(w)(c) for w in ws]))
            else:
                bool_list.append(p(ws)(c))
        return all(bool_list)

    if isinstance(ws,tuple):
        if ws[0] in Selection:
            return ps(ws)(c)
        else:
            return all([ps(w)(c) for w in ws])
    else:
        return ps(ws)(c)

#function for @Left and @Right
def at_POSI(POSI,ws,arg,c,option=None):
    if isinstance(ws,tuple):
        if ws[0] in Selection:
            return ws[1] in c.get_other_posi(POSI,arg[-1])[ws[0]]
        else:
            bool_list = []
            for w in ws:
                bool_list.append(at_POSI_0(POSI,arg,w,c,option))
            return all(bool_list)
    else:
        return at_POSI_0(POSI,arg,ws,c,option)

#function for @Left0 and @Right0
def at_POSI_0(POSI,arg,w,c,option=None):
    if arg not in ['ArgX','ArgY']:
        w,arg = arg,w
        if POSI == 'Left':
            POSI = 'Right'
        elif POSI == 'Right':
            POSI = 'Left'

    if isinstance(w,tuple) and w[0] not in Selection:
        return all([at_POSI_0(POSI,arg,ww,c,option) for ww in w])

    if w=='ArgY':
        w = c.obj
    elif w=='ArgX':
        w = c.subj

    if option==None:
        option = {'attr': 'word', 'range': -1, 'numAppear':1,'cmp':'nlt','onlyCount':False}  #For now, if onlyCount==True, then attr=='tokens'
    if isinstance(w,tuple):
        return w[1] in c.get_other_posi(POSI, arg[-1])[w[0]]
    else:                                                                                   #For now,option['attr']==tokens if and only if 'right before' is used or onlyCount==True, otherwise ==word
        w = w.lower()
        w_split = w.split()
        while '' in w_split:
            w_split.remove('')
        if option == 'Direct':
            range_ = len(w_split)
            info = [token.lower() for token in c.get_other_posi(POSI, arg[-1])['tokens']]
            if POSI == 'Left':
                st = max(0, len(info) - range_)
                info = info[st:]
            elif POSI == 'Right':
                ed = min(len(info) - 1, range_-1)
                info = info[:ed + 1]
            else:
                raise ValueError
            # print(info)
            if info==w_split:
                return True
            else:
                return False
        if option['range']==-1:
            info = [token.lower() for token in c.get_other_posi(POSI, arg[-1])['tokens']]
        else:                                                                               #For now, if range!=-1 then attr == 'tokens'
            info = [token.lower() for token in c.get_other_posi(POSI, arg[-1])[option['attr']]]
            range_ = option['range']
            if POSI == 'Left':
                st = max(0, len(info) - range_)
                info = info[st:]
            elif POSI=='Right':
                ed = min(len(info) - 1, range_-1)
                info = info[:ed + 1]
            else:
                count_posi = c.get_other_posi(POSI, arg[-1])['POSI']
                st = max(0,count_posi-range_)
                ed = min(len(info),count_posi+1+range_)
                info = info[st:ed+1]
        if option['onlyCount']:
            return compare[option['cmp']](len(info),option['numAppear'])
        else:
            return compare[option['cmp']](count_sublist(info,w_split),option['numAppear'])


#function for @Between

def at_between(w,c,option=None,a=None):
    if w=='ArgY':
        w = c.obj
    elif w=='ArgX':
        w = c.subj
    if option==None:
        option = {'attr': 'word', 'numAppear':1,'cmp':'nlt','onlyCount':False}                #For now, if onlyCount==True, then attr=='tokens'
    if isinstance(w,tuple):
        return w[1] in c.get_mid()[w[0]]
    else:                                                                                   #For now,option['attr']==tokens if and only if  onlyCount==True, otherwise ==word
        w = w.lower()
        w_split = w.split()
        while '' in w_split:
            w_split.remove('')
        info = [token.lower() for token in c.get_mid()['tokens']]
        if option['onlyCount']:
            return compare[option['cmp']](len(info),option['numAppear'])
        else:
            # print(info, w,info.count(w),type(info.count(w)),option['numAppear'],option['eq'],compare[option['eq']](info.count(w),option['numAppear']))
            return compare[option['cmp']](count_sublist(info,w_split),option['numAppear'])


#function for counting
def at_lessthan(funcx,nouny,w,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'lt','onlyCount':onlyCount})(c)                #There are less than 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(c)      #the word 'x' is less than 3 words before OBJ

def at_atmost(funcx,nouny,w,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'nmt','onlyCount':onlyCount})(c)                #There are at most 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(c)      #the word 'x' is at most 3 words before OBJ

def at_atleast(funcx,nouny,w,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'nlt','onlyCount':onlyCount})(c)             #There are at least 3 words before OBJ
    else:
        return funcx(w,{'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(c)      #the word 'x' is no less than 3 words before OBJ

def at_morethan(funcx,nouny,w,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'mt','onlyCount':onlyCount})(c)                #There are more than 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(c)    #the word 'x' is more than 3 words before OBJ


#function for @In0

def at_In0(arg,w,c):
    assert arg=='Sentence'
    if isinstance(w,tuple):
        return w in c.ner
    else:
        w = w.lower().split()
        info = [token.lower() for token in c.token]
        return count_sublist(info,w)>0

def at_WordCount(nounNum,nouny,F,c):
    if isinstance(nouny,tuple):
        return all([F(noun, option={'attr': 'word', 'range': -1, 'numAppear': 1, 'cmp': 'nlt', 'onlyCount': False})(c) for noun in nouny]) and F(nouny[0],option={'attr': 'tokens','range': -1,'numAppear':sum([len(noun.split()) for noun in nouny]),'cmp': 'eq','onlyCount': True})(c)
    else:
        return F(nouny,option={'attr':'word','range':-1,'numAppear':1,'cmp':'nlt','onlyCount':False})(c) and F(nouny,option={'attr':'tokens','range':-1,'numAppear':len(nouny.split()),'cmp':'eq','onlyCount':True})(c)

ops={
    ".root":lambda xs:lambda c:all([x(c) for x in xs]) if type(xs)==tuple else xs(c),
    "@Word":lambda x:x,
    "@Is":lambda ws,p: lambda c: IsFunc(ws,p,c),
    "@between": lambda a: lambda w,option=None:lambda c:at_between(w,c,option,a),
    "@And": lambda x,y:merge(x,y),
    "@Num": lambda x,y:{'attr':y,'num':int(x)},
    "@LessThan":lambda funcx,nouny:lambda w:lambda c:at_lessthan(funcx,nouny,w,c),
    "@AtMost":lambda funcx,nouny:lambda w:lambda c:at_atmost(funcx,nouny,w,c),
    "@AtLeast":lambda funcx,nouny:lambda w:lambda c:at_atleast(funcx,nouny,w,c),
    "@MoreThan":lambda funcx,nouny:lambda w:lambda c:at_morethan(funcx,nouny,w,c),
    "@WordCount":lambda nounNum,nouny,F:lambda useless:lambda c:at_WordCount(nounNum,nouny,F,c),

    "@NumberOf":lambda x,f:[x,f],
    "@LessThan1":lambda nounynum:lambda x:lambda c:at_lessthan(x[1],{'attr':x[0],"num":int(nounynum)},'There',c),
    "@AtMost1":lambda nounynum:lambda x:lambda c:at_atmost(x[1],{'attr':x[0],"num":int(nounynum)},'There',c),
    "@AtLeast1":lambda nounynum:lambda x:lambda c:at_atleast(x[1],{'attr':x[0],"num":int(nounynum)},'There',c),
    "@MoreThan1":lambda nounynum:lambda x:lambda c:at_morethan(x[1],{'attr':x[0],"num":int(nounynum)},'There',c),


    "@In0":lambda arg: lambda w:lambda c:at_In0(arg,w,c),
    #By
    "@By":lambda x,f,z:lambda c: f(x,{'attr': z['attr'], 'range': z['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': False})(c),
    # is xx Arg
    "@Left0":lambda arg: lambda w,option=None:lambda c:at_POSI_0('Left',arg,w,c,option),
    "@Right0":lambda arg: lambda w,option=None:lambda c:at_POSI_0('Right',arg,w,c,option),
    "@Range0":lambda arg: lambda w,option=None:lambda c:at_POSI_0('Range',arg,w,c,option),

    "@Left":lambda arg, ws,option=None:lambda c:at_POSI('Left',ws,arg,c,option),
    "@Right":lambda arg, ws,option=None:lambda c:at_POSI('Right',ws,arg,c,option),

    "@Direct":lambda func:lambda w:lambda c: func(w,'Direct')(c),

    "@StartsWith": lambda x,y: lambda c: c.with_(x[-1],'starts',y),
    "@EndsWith": lambda x,y: lambda c:c.with_(x[-1],'ends',y),
}


syntax = {
    "@LOC": ('NER','LOCATION'),
    "@Date":('NER','DATE'),
    "@Num":('NER','NUMBER'),
    "@Org":('NER','ORGANIZATION'),
    "@Norp":('NER','Norp'),
    '@PER':('NER','PERSON')
}

lexicon_head = ''':- S,NP,N,PP
        VP :: S\\NP
        Det :: NP/N
        Adj :: N/N'''

raw_lexicon =''':- S,NP,N,PP
        VP :: S\\NP
        Det :: NP/N
        Adj :: N/N
        arg => NP {None}
        #$True => (S\\VP)/PP {None}
        #$False => (S\\VP)/PP {None}
        $And => var\\.,var/.,var {\\x y.'@And'(x,y)}
        $Or => var\\.,var/.,var {\\x y.'@Or'(x,y)}
        $Not => (S\\NP)\\(S\\NP) {None}
        $Not => (S\\NP)/(S\\NP) {None}
        $All => NP/N {None}
        $All => NP {None}
        $All => NP/NP {None}
        $Any => NP/N {None}
        $None => N {None}
        #$Is => (S\\NP)/NP {\\y x.'@Is'(x,y)}
        #$Is => (S\\NP)/(S\\NP) {\\y x.'@Is'(x,y)}
        $Is => (S\\NP)/PP {\\y x.'@Is'(x,y)}   # word 'a' occurs between <S> and <O>
        $Is => (S\\NP)\\PP {\\y x.'@Is'(x,y)}  # between <S> and <O> occurs word 'a'
        $Is => (S\\PP)\\NP {\\x y.'@Is'(x,y)}  # between <S> and <O> word 'a' occurs
        $Exists => S\\NP/PP {\\y x.'@Is'(x,y)}
        #$Exists => S\\NP {None}
        $Int => Adj {None} #There are no words between <S> and <O>
        $AtLeastOne => NP/N {None}
        #$Equals => (S\\NP)/NP {None}
        #$NotEquals => (S\\NP)/NP {None}
        
        
        $LessThan => PP/PP/N {\\x y.'@LessThan'(y,x)} #There are less than 3 words between <S> and <O>   
        $AtMost => PP/PP/N {\\x y.'@AtMost'(y,x)} #There are at most 3 words between <S> and <O>
        $AtLeast => PP/PP/N {\\x y.'@AtLeast'(y,x)} #same as above
        $MoreThan => PP/PP/N {\\x y.'@MoreThan'(y,x)} #same as above
        
        $LessThan => PP/N {\\x.'@LessThan1'(y,x)} #number of words between X and Y is less than 7.
        $AtMost => PP/N {\\x.'@AtMost1'(y,x)} 
        $AtLeast => PP/N {\\x.'@AtLeast1'(y,x)}   #same as above
        $MoreThan => PP/N {\\x.'@MoreThan1'(y,x)} #same as above

        #$In => S\\NP/NP {None} 
        $In => PP/NP {\\x.'@In0'(x)} 
        $Contains => S\\NP/NP {None} #The sentence contains two words
        $Separator => var\\.,var/.,var {\\x y.'@And'(x,y)} #connection between two words
        #$Processive => NP/N\\N {None}
        #$Count => N {None}
        #$Tuple => N {None}
        #$ArgXListAnd => NP {None}
        $EachOther => N {None}
        $Token => N {\\x.'@Word'(x)}
        $Word => NP/N {\\x.'@Word'(x)}
        $Word => NP/NP {\\x.'@Word'(x)}
        
        $Word => N {'tokens'} #There are no more than 3 words between <S> and <O>
        $Word => NP {'tokens'} #There are no more than 3 words between <S> and <O>
        
        $Char => N {None} #same as above
        #$Lower => Adj {None}
        #$Capital => Adj {None}
        $StartsWith => S\\NP/NP {\\y x.'@StartsWith'(x,y)}
        $EndsWith => S\\NP/NP {\\y x.'@EndsWith'(x,y)}
        $Left => PP/NP {\\x.'@Left0'(x)} # the word 'a' is before <S>
        $Left => (S\\NP)/NP {\\y x.'@Left'(y,x)}  #Precedes
        $Right => PP/NP {\\x.'@Right0'(x)}# the word 'a' ia after <S>
        $Right => (S\\NP)/NP {\\y x.'@Right'(y,x)} 
        #$Within => ((S\\NP)\\(S\\NP))/NP {None} # the word 'a' is within 2 words after <S>
        #$Within => (NP\\NP)/NP {None}
        $Within => PP/PP/N {\\x y.'@AtMost'(y,x)} #Does Within has other meaning.
        $Sentence => NP {'Sentence'}
        
        $Between => (S/S)/NP {\\x y.'@between'(x,y)}
        $Between => S/NP {\\x.'@between'(x)}
        $Between => PP/NP {\\x.'@between'(x)}
        $Between => (NP\\NP)/NP {\\x y.'@between'(x,y)}
        
        $PersonNER => NP {'@PER'}
        $LocationNER => NP {'@LOC'}
        $DateNER => NP {'@Date'}
        $NumberNER => NP {'@Num'}
        $OrganizationNER => NP {'@Org'}
        $NorpNER => NP {'@Norp'}
        $ArgX => NP {'ArgX'}
        $ArgY => NP {'ArgY'}
        #$will => S\\NP/VP {None}
        #$Which => (NP\\NP)/(S/NP) {None}
        #$might => S\\NP/VP {None}
        $that => NP/N {None}
        #$that => (N\\N)/(S/NP) {None} #same as which
        $Apart => (S/PP)\\NP {None}
        $Direct => PP/PP {\\x.'@Direct'(x)} # the word 'a' is right before <S>   
        $Direct => (S\\NP)/PP {\\y x.'@Is'(x,'@Direct'(y))}
        $Last => Adj {None}
        $There => NP {'There'}
        $By => S\\NP\\PP/NP {\\z f x.'@By'(x,f,z)} #precedes sth by 10 chatacters       
        $By => (S\\NP)\\PP/(PP/PP) {\\F x y.'@Is'(y,F(x))} #precedes sth by no more than10 chatacters        
        $By => PP\\PP/(PP/PP) {\\F x. F(x)} #occurs before by no...
        
        $Numberof => NP/PP/NP {\\x F.'@NumberOf'(x,F)}
        
        $Of => PP/NP {\\x.'@Range0'(x)} # the word 'x' is at most 3 words of Y     
        $Of => NP/NP {\\x.x} #these two are designed to solve problems like $Is $Left $Of and $Is $Left
        $Of => N/N {\\x.x}
        $Char => NP/N {None}
        $ArgX => N {'ArgX'}
        $ArgY => N {'ArgY'}
        $Link => (S\\NP)/NP {\\x y.'@Is'(y,'@between'(x))}
        $SandWich => (S\\NP)/NP {\\x y.'@Is'(x,'@between'(y))}
        $The => N/N {\\x.x}
        $The =>NP/NP {\\x.x}
        '''


In [120]:
unmatch_count_score = 0.5
unmatch_count_dist = 7
unmatch_match_score = 0.5
unmatch_match_dist = 7


Selection = ['NER']
compare_soft={                   #eq mt(more than) lt(less than) nmt(no more than) nlt(no less than)
    'eq':lambda a,b:tf.math.maximum(tf.cast(tf.equal(a,b),tf.float32),tf.cast(tf.logical_and(tf.greater_equal(a,b-unmatch_count_dist),tf.less_equal(a,b+unmatch_count_dist)),tf.float32)*unmatch_count_score),
    'mt':lambda a,b:tf.math.maximum(tf.cast(tf.greater(a,b),tf.float32),tf.cast(tf.greater(a,b-unmatch_count_dist),tf.float32)*unmatch_count_score),
    'lt':lambda a,b:tf.math.maximum(tf.cast(tf.less(a,b),tf.float32),tf.cast(tf.less(a,b+unmatch_count_dist),tf.float32)*unmatch_count_score),
    'nmt':lambda a,b:tf.math.maximum(tf.cast(tf.less_equal(a,b),tf.float32),tf.cast(tf.less_equal(a,b+unmatch_count_dist),tf.float32)*unmatch_count_score),
    'nlt':lambda a,b:tf.math.maximum(tf.cast(tf.greater_equal(a,b),tf.float32),tf.cast(tf.greater_equal(a,b-unmatch_count_dist),tf.float32)*unmatch_count_score),
}

#c: Tensor [seqlen seqlen 2 2] : sentence,ner,subj_posi,obj_posi,subj,obj
def get_mid(attr,mask_mat,c):
    subj_posi = c[:,-4:-3]
    obj_posi = c[:,-3:-2]
    seqlen = (c.get_shape()[1]-4)//2
    tokens = c[:,0:seqlen]
    ner = c[:,seqlen:seqlen*2]
    st_posi = tf.math.minimum(subj_posi,obj_posi)
    ed_posi = subj_posi+obj_posi-st_posi
    mask = tf.gather_nd(mask_mat,tf.concat([st_posi+1,ed_posi-1],axis=1))
    ner = ner*mask
    tokens = tokens*mask
    res = tf.cond(attr=='NER',lambda:ner,lambda:tokens)
    return res

def get_other_posi(POSI,attr,arg,mask_mat,c):
    assert POSI=='Left' or POSI=='Right'
    batch_size = c.get_shape()[0]
    subj_posi = c[:, -4:-3]
    obj_posi = c[:, -3:-2]
    seqlen = (c.get_shape()[1] - 4) // 2
    tokens = c[:, 0:seqlen]
    ner = c[:, seqlen:seqlen * 2]
    posi = tf.cond(arg=='ArgY',lambda:obj_posi,lambda:subj_posi)

    mask = tf.cond(POSI=='Left',lambda:tf.gather_nd(mask_mat,tf.concat([0*posi,posi-1],axis=1)),
                   lambda:tf.gather_nd(mask_mat,tf.concat([posi+1,(0*posi+1)*(seqlen-1)],axis=1)))

    new_ner = mask*ner
    new_tokens = mask*tokens
    res = tf.cond(attr=='NER',lambda:new_ner,lambda:new_tokens)
    return res

def get_range(attr,arg,range_,mask_mat,c):
    subj_posi = c[:, -4:-3]
    obj_posi = c[:, -3:-2]
    seqlen = (c.get_shape()[1] - 4) // 2
    tokens = c[:, 0:seqlen]
    ner = c[:, seqlen:seqlen * 2]
    posi = tf.cond(arg == 'ArgY', lambda: obj_posi, lambda: subj_posi)
    res = tf.cond(attr == 'NER', lambda: ner, lambda: tokens)
    mask = tf.gather_nd(mask_mat,tf.concat([tf.math.maximum(0,posi-range_),tf.math.minimum(seqlen-1,posi+range_)],axis=1))
    res = res*mask
    return res

def In(w,seq):
    assert w in NER2ID
    idx = NER2ID[w]
    eq = tf.equal(seq,idx)
    sum = tf.reshape(tf.reduce_sum(eq,axis=1),[1,-1])
    return tf.cast(sum,tf.bool)


def merge_soft(x,y):
    if type(x)!= tuple:
        x = [x]
    if type(y)!= tuple:
        y = [y]
    x = list(x)
    y = list(y)
    return tuple(x+y)

#function for $Is
def IsFunc_soft(ws,ps,label_mat,keyword_dict,mask_mat,c):
    if isinstance(ps,tuple):
        bool_list  = []
        for p in ps:
            if isinstance(ws, tuple):
                if ws[0] in Selection:
                    bool_list.append(p(ws)(label_mat,keyword_dict,mask_mat)(c))
                else:
                    bool_list.append(tf.maximum(sum([p(w)(label_mat,keyword_dict,mask_mat)(c) for w in ws])-len(ws)+1,0.0))
            else:
                bool_list.append(p(ws)(label_mat,keyword_dict,mask_mat)(c))
        return tf.maximum(sum(bool_list)-len(bool_list)+1,0.0)

    if isinstance(ws,tuple):
        if ws[0] in Selection:
            return ps(ws)(label_mat,keyword_dict,mask_mat)(c)
        else:
            return tf.maximum(sum([ps(w)(label_mat,keyword_dict,mask_mat)(c) for w in ws])-len(ws)+1,0.0)
    else:
        return ps(ws)(label_mat,keyword_dict,mask_mat)(c)

#function for @Left and @Right
def at_POSI_soft(POSI,ws,arg,label_mat,keyword_dict,mask_mat,c,option=None):
    if isinstance(ws,tuple):
        if ws[0] in Selection:
            assert POSI=='Left' or POSI=='Right' or POSI=='Range'
            if POSI=='Left' or POSI=='Right':
                score_raw = tf.cast(In(ws[1],get_other_posi(POSI,ws[0],arg,mask_mat,c)),tf.float32)
                return score_raw
            else:
                score_raw =  tf.cast(In(ws[1], get_range(ws[0], arg, option['range'],mask_mat,c)), tf.float32)
                return score_raw
        else:
            bool_list = []
            for w in ws:
                bool_list.append(at_POSI_0_soft(POSI,arg,w,label_mat,keyword_dict,mask_mat,c,option))
            return tf.maximum(sum(bool_list)-len(bool_list)+1,0.0)
    else:
        return at_POSI_0_soft(POSI,arg,ws,label_mat,keyword_dict,mask_mat,c,option)

#function for @Left0 and @Right0
def at_POSI_0_soft(POSI,arg,w,label_mat,keyword_dict,mask_mat,c,option=None):
    if arg not in ['ArgX','ArgY']:
        w,arg = arg,w
        if POSI == 'Left':
            POSI = 'Right'
        elif POSI == 'Right':
            POSI = 'Left'

    if isinstance(w,tuple) and w[0] not in Selection:
        return tf.maximum(sum([at_POSI_0_soft(POSI,arg,ww,label_mat,keyword_dict,mask_mat,c,option) for ww in w])-len(w)+1,0.0)


    if option==None:
        option = {'attr': 'word', 'range': -1, 'numAppear':1,'cmp':'nlt','onlyCount':False}  #For now, if onlyCount==True, then attr=='tokens'
    if isinstance(w,tuple):
        assert POSI == 'Left' or POSI == 'Right' or POSI == 'Range'
        if POSI == 'Left' or POSI == 'Right':
            score_raw = tf.cast(In(ws[1], get_other_posi(POSI, ws[0], arg, mask_mat,c)), tf.float32)
            return score_raw
        else:
            score_raw = tf.cast(In(ws[1], get_range(ws[0], arg, option['range'], mask_mat,c)), tf.float32)
            return score_raw
    else:                                                                                   #For now,option['attr']==tokens if and only if 'right before' is used or onlyCount==True, otherwise ==word
        if option == 'Direct':
            w_split = w.split()
            while '' in w_split:
                w_split.remove('')
            range_ = len(w_split)

            subj_posi = c[:, -4:-3]
            obj_posi = c[:, -3:-2]
            seqlen = (c.get_shape()[1] - 4) // 2

            if arg=='ArgY':
                arg_posi = obj_posi
            else:
                arg_posi = subj_posi

            if POSI == 'Left':
                st = tf.math.maximum(0, arg_posi - range_)
                position = tf.concat([st,arg_posi-1],axis=1)
                unmatch_position = tf.concat([tf.math.maximum(0,arg_posi - range_ - unmatch_match_dist), arg_posi - 1], axis=1)
            elif POSI == 'Right':
                st = arg_posi
                ed = tf.math.minimum(seqlen - 1, arg_posi+range_)
                position = tf.concat([st+1,ed],axis=1)
                unmatch_position = tf.concat([st + 1, tf.math.minimum(seqlen - 1,st+range_+unmatch_match_dist)], axis=1)
            else:
                raise ValueError
            if w in ['ArgX','ArgY']:
                if w=='ArgX':
                    tar = c[:,-4:-3]
                    src = c[:,-3:-2]
                else:
                    src = c[:,-4:-3]
                    tar = c[:,-3:-2]
                if POSI=='Left':
                    score_raw = tf.reshape(tf.cast(tf.equal(src-tar,1),tf.float32),[1,-1])
                    return score_raw
                else:
                    score_raw = tf.reshape(tf.cast(tf.equal(tar-src, 1),tf.float32),[1,-1])
                    return score_raw
            else:
                mask_raw = tf.cast(tf.gather_nd(mask_mat, position), tf.float32)
                mask_raw = mask_raw+(tf.cast(tf.gather_nd(mask_mat, unmatch_position), tf.float32)-mask_raw)*unmatch_match_score
                return tf.reshape(tf.reduce_max(label_mat[:, :, keyword_dict[w]] * (mask_raw),axis=1), [1, -1])

        if option['range']==-1:
            assert POSI=='Left' or POSI=='Right'
            batch_size = c.get_shape()[0]
            subj_posi = c[:, -4:-3]
            obj_posi = c[:, -3:-2]
            seqlen = (c.get_shape()[1] - 4) // 2
            if arg == 'ArgY':
                arg_posi = obj_posi
            else:
                arg_posi = subj_posi

            if POSI=='Left':
                position = tf.concat([0*arg_posi, arg_posi - 1], axis=1)
                unmatch_position = position
            else:
                position = tf.concat([arg_posi + 1, (0*arg_posi+1) * (seqlen - 1)],axis=1)
                unmatch_position = position

        else:                                                                               #For now, if range!=-1 then attr == 'tokens'
            subj_posi = c[:, -4:-3]
            obj_posi = c[:, -3:-2]
            seqlen = (c.get_shape()[1] - 4) // 2
            if arg == 'ArgY':
                arg_posi = obj_posi
            else:
                arg_posi = subj_posi

            range_ = option['range']

            if POSI == 'Left':
                st = tf.math.maximum(0, arg_posi - range_)
                position = tf.concat([st, arg_posi-1],axis=1)
                unmatch_position = tf.concat([tf.math.maximum(0, arg_posi - range_ - unmatch_match_dist), arg_posi - 1],axis=1)
            elif POSI=='Right':
                st = arg_posi
                ed = tf.math.minimum(seqlen - 1, arg_posi+range_)
                position = tf.concat([arg_posi+1, ed],axis=1)
                unmatch_position = tf.concat([st + 1, tf.math.minimum(seqlen - 1, st + range_ + unmatch_match_dist)],axis=1)
            else:
                st = tf.math.maximum(0,arg_posi-range_)
                ed = tf.math.minimum(seqlen-1,arg_posi+range_)
                position=tf.concat([st,ed],axis=1)
                unmatch_position = tf.concat([tf.math.maximum(0, arg_posi - range_ - unmatch_match_dist), tf.math.minimum(seqlen - 1, st + range_ + unmatch_match_dist)], axis=1)

        if option['onlyCount']:
            score_raw = tf.reshape(tf.cast(compare_soft[option['cmp']](position[:,1]-position[:,0],option['numAppear']),tf.float32),[1,-1])
            return score_raw
        else:
            assert option['cmp']=='nlt' and option['numAppear']==1
            if w in ['ArgX','ArgY']:
                if w=='ArgX':
                    tar = c[:,-4]
                else:
                    tar = c[:,-3]
                score_raw = tf.reshape(tf.cast(tf.logical_and(tar>=position[:,0],tar<=position[:,1]),tf.float32),[1,-1])
                return score_raw
            else:
                mask_raw = tf.cast(tf.gather_nd(mask_mat, position),tf.float32)
                mask_raw = mask_raw+(tf.cast(tf.gather_nd(mask_mat, unmatch_position),tf.float32)-mask_raw)*unmatch_match_score
                return tf.reshape(tf.reduce_max(label_mat[:, :, keyword_dict[w]] * (mask_raw),axis=1), [1, -1])



#function for @Between

def at_between_soft(w,label_mat,keyword_dict,mask_mat,c,option=None):
    if option==None:
        option = {'attr': 'word', 'numAppear':1,'cmp':'nlt','onlyCount':False}                #For now, if onlyCount==True, then attr=='tokens'
    if isinstance(w,tuple):
        score_raw = tf.cast(In(w[1],get_mid(w[0],mask_mat,c)),tf.float32)
        return score_raw
    else:                                                                                   #For now,option['attr']==tokens if and only if  onlyCount==True, otherwise ==word
        if option['onlyCount']:
            score_raw =  tf.reshape(tf.cast(compare_soft[option['cmp']](tf.abs(c[:,-4]-c[:,-3])-1,option['numAppear']),tf.float32),[1,-1])
            return score_raw
        else:
            assert option['cmp'] == 'nlt' and option['numAppear'] == 1
            l_posi = tf.math.minimum(c[:,-4],c[:,-3])
            g_posi = c[:,-4]+c[:,-3]-l_posi
            if w in ['ArgX','ArgY']:
                if w=='ArgX':
                    tar = c[:,-4]
                else:
                    tar = c[:,-3]
                score_raw = tf.reshape(tf.cast(tf.logical_and(tar>l_posi,tar<g_posi),tf.float32),[1,-1])
                return score_raw
            else:
                l_posi = tf.reshape(l_posi,[-1,1])+1
                g_posi = tf.reshape(g_posi, [-1, 1])-1
                position = tf.concat([l_posi, g_posi],axis=1)
                mask_raw = tf.cast(tf.gather_nd(mask_mat, position),tf.float32)
                seqlen = (c.get_shape()[1] - 4) // 2
                return tf.reshape(tf.reduce_max(label_mat[:, :, keyword_dict[w]] * (mask_raw),axis=1), [1, -1])



#function for counting
def at_lessthan_soft(funcx,nouny,w,label_mat,keyword_dict,mask_mat,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'lt','onlyCount':onlyCount})(label_mat,keyword_dict,mask_mat)(c)                #There are less than 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(label_mat,keyword_dict,mask_mat)(c)      #the word 'x' is less than 3 words before OBJ

def at_atmost_soft(funcx,nouny,w,label_mat,keyword_dict,mask_mat,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'nmt','onlyCount':onlyCount})(label_mat,keyword_dict,mask_mat)(c)                #There are at most 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(label_mat,keyword_dict,mask_mat)(c)      #the word 'x' is at most 3 words before OBJ

def at_atleast_soft(funcx,nouny,w,label_mat,keyword_dict,mask_mat,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'nlt','onlyCount':onlyCount})(label_mat,keyword_dict,mask_mat)(c)             #There are at least 3 words before OBJ
    else:
        return funcx(w,{'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(label_mat,keyword_dict,mask_mat)(c)      #the word 'x' is no less than 3 words before OBJ

def at_morethan_soft(funcx,nouny,w,label_mat,keyword_dict,mask_mat,c):
    if w=='There':
        onlyCount=True
    else:
        onlyCount = False
    if onlyCount:
        return funcx(w,{'attr':nouny['attr'],'range':-1,'numAppear':nouny['num'],'cmp':'mt','onlyCount':onlyCount})(label_mat,keyword_dict,mask_mat)(c)                #There are more than 3 words before OBJ
    else:
        return funcx(w, {'attr': nouny['attr'], 'range': nouny['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': onlyCount})(label_mat,keyword_dict,mask_mat)(c)    #the word 'x' is more than 3 words before OBJ


#function for @In0

def at_In0_soft(arg,w,label_mat,keyword_dict,mask_mat,c):
    assert arg=='Sentence'                                                                                                          #Right?
    if isinstance(w,tuple):
        seqlen = (c.get_shape()[1] - 4) // 2
        score_raw = tf.cast(In(w,c[:,seqlen:seqlen*2]),tf.float32)
        return score_raw
    else:
        seqlen = (c.get_shape()[1] - 4) // 2
        return tf.reshape(tf.reduce_max(label_mat[:, :, keyword_dict[w]], axis=1), [1, -1])
        # return w in c.sentence

def at_WordCount_soft(nounNum,nouny,F,label_mat,keyword_dict,mask_mat,c):
    if isinstance(nouny,tuple):
        return tf.maximum(tf.maximum(sum([F(noun, option={'attr': 'word', 'range': -1, 'numAppear': 1, 'cmp': 'nlt', 'onlyCount': False})(label_mat,keyword_dict,mask_mat)(c) for noun in nouny])-len(nouny)+1,0.0)+F(nouny[0],option={'attr': 'tokens','range': -1,'numAppear':sum([len(noun.split()) for noun in nouny]),'cmp': 'eq','onlyCount': True})(label_mat,keyword_dict,mask_mat)(c)-1,0.0)
    else:
        return tf.maximum(F(nouny, option={'attr':'word','range':-1,'numAppear':1,'cmp':'nlt','onlyCount':False})(label_mat,keyword_dict,mask_mat)(c)+F(nouny, option={'attr':'tokens','range':-1,'numAppear':len(nouny.split()),'cmp':'eq','onlyCount':True})(label_mat,keyword_dict,mask_mat)(c)-1,0.0)

ops_soft={
    ".root": lambda xs: lambda label_mat,keyword_dict,mask_mat:lambda c: tf.maximum(sum([x(label_mat,keyword_dict,mask_mat)(c) for x in xs])-len(xs)+1,0.0) if type(xs) == tuple else xs(label_mat,keyword_dict,mask_mat)(c),
    "@Word": lambda x: x,
    "@Is": lambda ws, p: lambda label_mat,keyword_dict,mask_mat:lambda c: IsFunc_soft(ws, p,label_mat,keyword_dict,mask_mat, c),
    "@between": lambda a: lambda w, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_between_soft(w, label_mat,keyword_dict,mask_mat,c, option),
    "@And": lambda x, y: merge_soft(x, y),
    "@Num": lambda x, y: {'attr': y, "num": int(x)},
    "@LessThan": lambda funcx, nouny: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: at_lessthan_soft(funcx, nouny, w, label_mat,keyword_dict,mask_mat,c),
    "@AtMost": lambda funcx, nouny: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: at_atmost_soft(funcx, nouny, w, label_mat,keyword_dict,mask_mat,c),
    "@AtLeast": lambda funcx, nouny: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: at_atleast_soft(funcx, nouny, w, label_mat,keyword_dict,mask_mat,c),
    "@MoreThan": lambda funcx, nouny: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: at_morethan_soft(funcx, nouny, w, label_mat,keyword_dict,mask_mat,c),
    "@WordCount": lambda nounNum, nouny, F:lambda useless: lambda label_mat,keyword_dict,mask_mat:lambda c: at_WordCount_soft(nounNum,nouny,F,label_mat,keyword_dict,mask_mat,c),

    "@NumberOf": lambda x, f: [x, f],
    "@LessThan1": lambda nounynum: lambda x: lambda label_mat,keyword_dict,mask_mat:lambda c: at_lessthan_soft(x[1], {'attr': x[0], "num": int(nounynum)}, 'There',label_mat,keyword_dict,mask_mat,c),
    "@AtMost1": lambda nounynum: lambda x: lambda label_mat,keyword_dict,mask_mat:lambda c: at_atmost_soft(x[1], {'attr': x[0], "num": int(nounynum)}, 'There', label_mat,keyword_dict,mask_mat,c),
    "@AtLeast1": lambda nounynum: lambda x: lambda label_mat,keyword_dict,mask_mat:lambda c: at_atleast_soft(x[1], {'attr': x[0], "num": int(nounynum)}, 'There',label_mat,keyword_dict,mask_mat,c),
    "@MoreThan1": lambda nounynum: lambda x: lambda label_mat,keyword_dict,mask_mat:lambda c: at_morethan_soft(x[1], {'attr': x[0], "num": int(nounynum)}, 'There',label_mat,keyword_dict,mask_mat,c),

    "@In0": lambda arg: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: at_In0_soft(arg, w, label_mat,keyword_dict,mask_mat,c),

    "@By": lambda x, f, z: lambda label_mat,keyword_dict,mask_mat:lambda c: f(x, {'attr': z['attr'], 'range': z['num'], 'numAppear': 1, 'cmp': 'nlt','onlyCount': False})(label_mat,keyword_dict,mask_mat)(c),

    "@Left0": lambda arg: lambda w, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_POSI_0_soft('Left', arg, w, label_mat,keyword_dict,mask_mat,c, option),

    "@Right0": lambda arg: lambda w, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_POSI_0_soft('Right', arg, w, label_mat,keyword_dict,mask_mat,c, option),

    "@Range0": lambda arg: lambda w, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_POSI_0_soft('Range', arg, w, label_mat,keyword_dict,mask_mat,c, option),

    "@Left": lambda arg, ws, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_POSI_soft('Left', ws, arg, label_mat,keyword_dict,mask_mat,c, option),
    "@Right": lambda arg, ws, option=None: lambda label_mat,keyword_dict,mask_mat:lambda c: at_POSI_soft('Right', ws, arg, label_mat,keyword_dict,mask_mat,c, option),

    "@Direct": lambda func: lambda w: lambda label_mat,keyword_dict,mask_mat:lambda c: func(w, 'Direct')(label_mat,keyword_dict,mask_mat)(c)
}

In [86]:
# mini_sent = """
# The phrase \"'s grandmother\" occurs between SUBJ and OBJ
# """

longer_sent = """
The phrase \"'s grandmother\" occurs between SUBJ and OBJ and there are no more than four words between SUBJ and OBJ
"""

In [87]:
sent = longer_sent

In [88]:
pre_process = CCG.utils.pre_process_sent(sent)

In [89]:
pre_process

['The',
 'phrase',
 '"\'s grandmother"',
 'occurs',
 'between',
 'SUBJ',
 'and',
 'OBJ',
 'and',
 'there',
 'are',
 '$AtMost',
 'four',
 'words',
 'between',
 'SUBJ',
 'and',
 'OBJ']

In [90]:
sent_tokenized = CCG.utils.print_tokenized(pre_process)

In [91]:
sent_tokenized

[['$The',
  '$Word',
  '"\'s grandmother"',
  '$Is',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY',
  '$And',
  '$There',
  '$Is',
  '$AtMost',
  "'4'",
  '$Word',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY']]

In [92]:
from nltk.ccg import chart, lexicon
# from stanfordcorenlp import StanfordCoreNLP
import CCG.constant

In [93]:
sent_tokenized[0]

['$The',
 '$Word',
 '"\'s grandmother"',
 '$Is',
 '$Between',
 '$ArgX',
 '$And',
 '$ArgY',
 '$And',
 '$There',
 '$Is',
 '$AtMost',
 "'4'",
 '$Word',
 '$Between',
 '$ArgX',
 '$And',
 '$ArgY']

In [94]:
CCG.utils.new_predicate(sent_tokenized)

["''s grandmother'", "'4'"]

In [95]:
raw_lexicon = CCG.constant.raw_lexicon
predicates = CCG.utils.new_predicate(sent_tokenized)
raw_lexicon = CCG.utils.add_new_predicate(predicates, raw_lexicon)

In [96]:
raw_lexicon

":- S,NP,N,PP\n        VP :: S\\NP\n        Det :: NP/N\n        Adj :: N/N\n        arg => NP {None}\n        #$True => (S\\VP)/PP {None}\n        #$False => (S\\VP)/PP {None}\n        $And => var\\.,var/.,var {\\x y.'@And'(x,y)}\n        $Or => var\\.,var/.,var {\\x y.'@Or'(x,y)}\n        $Not => (S\\NP)\\(S\\NP) {None}\n        $Not => (S\\NP)/(S\\NP) {None}\n        $All => NP/N {None}\n        $All => NP {None}\n        $All => NP/NP {None}\n        $Any => NP/N {None}\n        $None => N {None}\n        #$Is => (S\\NP)/NP {\\y x.'@Is'(x,y)}\n        #$Is => (S\\NP)/(S\\NP) {\\y x.'@Is'(x,y)}\n        $Is => (S\\NP)/PP {\\y x.'@Is'(x,y)}   # word 'a' occurs between <S> and <O>\n        $Is => (S\\NP)\\PP {\\y x.'@Is'(x,y)}  # between <S> and <O> occurs word 'a'\n        $Is => (S\\PP)\\NP {\\x y.'@Is'(x,y)}  # between <S> and <O> word 'a' occurs\n        $Exists => S\\NP/PP {\\y x.'@Is'(x,y)}\n        #$Exists => S\\NP {None}\n        $Int => Adj {None} #There are no words between

In [97]:
lex = lexicon.fromstring(raw_lexicon, True)
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)

In [98]:
# raw_lexicon

In [99]:
for index_1 in range(len(sent_tokenized)):
    while [] in sent_tokenized[index_1]:
        sent_tokenized[index_1].remove([])
    for index_2 in range(len(sent_tokenized[index_1])):
        if sent_tokenized[index_1][index_2].startswith("'") or sent_tokenized[index_1][index_2].startswith("\""):
            sent_tokenized[index_1][index_2] = sent_tokenized[index_1][index_2].replace(' ','SPACE')
            sent_tokenized[index_1][index_2] = sent_tokenized[index_1][index_2].replace("\"","'")

In [100]:
sent_tokenized

[['$The',
  '$Word',
  "''sSPACEgrandmother'",
  '$Is',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY',
  '$And',
  '$There',
  '$Is',
  '$AtMost',
  "'4'",
  '$Word',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY']]

In [101]:
logic_forms = []

In [102]:
for i, e in enumerate(sent_tokenized):
    print(i)
    parses = list(parser.parse(e))
    logic_forms += parses

0


In [117]:
# logic_forms

In [111]:
semantics = []
for form in logic_forms:
    semantics.append(str(form.label()[0].semantics()))

In [112]:
set_semantics = list(set(semantics))
len(set_semantics)

2

In [113]:
set_semantics[0]

"'@And'('@Is'('There','@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('@And'('ArgY','ArgX'))))"

In [114]:
set_semantics[1]

"'@And'('@Is'('@And'('There','ArgY'),'@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('ArgX')))"

In [108]:
str(logic_forms[0].label()[0]._categ)

'S'

In [122]:
type(recurse_soft(set_semantics[1]))

str

In [110]:
# start = blah.start()
# print(start)

In [92]:
semantics = []
for form in logic_forms:
    semantics.append(str(form.label()[0].semantics()))

In [93]:
semantics[0]

"'@And'('@Is'('@And'('There','ArgY'),'@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('ArgX')))"

In [94]:
temp_tokens = str(logic_forms[0]).split("\n")

In [95]:
temp_tokens_1 = str(logic_forms[1]).split("\n")

In [96]:
for token in temp_tokens:
    if token not in temp_tokens_1:
        print(token)

((<nltk.ccg.lexicon.Token object at 0x133d2aba8>, '<')
  ((<nltk.ccg.lexicon.Token object at 0x134e07a20>, '>')
    ((<nltk.ccg.lexicon.Token object at 0x132f28320>, '<')
      ((<nltk.ccg.lexicon.Token object at 0x132f28cc0>, '>')
        ((<nltk.ccg.lexicon.Token object at 0x132f28b70>, '>')
          ((<nltk.ccg.lexicon.Token object at 0x132f28c88>, '>')
            ((<nltk.ccg.lexicon.Token object at 0x13455ba20>, 'Leaf')
              (<nltk.ccg.lexicon.Token object at 0x13455ba20>
                $AtMost))
            ((<nltk.ccg.lexicon.Token object at 0x132f28e80>, '>')
                (<nltk.ccg.lexicon.Token object at 0x133b9ec18> '4'))
              ((<nltk.ccg.lexicon.Token object at 0x133a8b438>, 'Leaf')
                (<nltk.ccg.lexicon.Token object at 0x133a8b438>
                  $Word))))


In [97]:
temp_tokens_1

["((<nltk.ccg.lexicon.Token object at 0x1341f0550>, '<')",
 "  ((<nltk.ccg.lexicon.Token object at 0x132f1d908>, '<')",
 "    ((<nltk.ccg.lexicon.Token object at 0x132f1d780>, '>')",
 "      ((<nltk.ccg.lexicon.Token object at 0x131a90eb8>, 'Leaf')",
 '        (<nltk.ccg.lexicon.Token object at 0x131a90eb8> $The))',
 "      ((<nltk.ccg.lexicon.Token object at 0x132f1d978>, '>')",
 "        ((<nltk.ccg.lexicon.Token object at 0x133cdcb70>, 'Leaf')",
 '          (<nltk.ccg.lexicon.Token object at 0x133cdcb70> $Word))',
 "        ((<nltk.ccg.lexicon.Token object at 0x131a902b0>, 'Leaf')",
 '          (<nltk.ccg.lexicon.Token object at 0x131a902b0>',
 "            ''sSPACEgrandmother'))))",
 "    ((<nltk.ccg.lexicon.Token object at 0x132f1dd68>, '>')",
 "      ((<nltk.ccg.lexicon.Token object at 0x1341a6128>, 'Leaf')",
 '        (<nltk.ccg.lexicon.Token object at 0x1341a6128> $Is))',
 "      ((<nltk.ccg.lexicon.Token object at 0x132f1dda0>, '>')",
 "        ((<nltk.ccg.lexicon.Token object

In [98]:
chart.printCCGDerivation(logic_forms[0])

AttributeError: 'list' object has no attribute 'pos'

In [34]:
for i, obj in enumerate(logic_forms):
    print(type(obj))
    if i > 30:
        break

<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>
<class 'nltk.tree.Tree'>


In [35]:
from nltk.draw import TreeView

In [178]:
TreeView(logic_forms[0])._cframe.print_to_file('output.ps')

In [40]:
tree_1 = logic_forms[0]

In [41]:
strtree_1.label()[0])

S {'@And'('@Is'('@And'('There','ArgY'),'@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('ArgX')))}


In [48]:
print(tree_2.label()[0])

S {'@And'('@Is'('@And'('There','ArgY'),'@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('ArgX')))}


In [46]:
tree_2.label()[0] 

In [51]:
initial_tree = str(logic_forms[0].label()[0].semantics())
for i, tree in enumerate(logic_forms):
    str_tree = str(tree.label()[0].semantics())
    if str_tree == initial_tree:
        print("yes")
    else:
        print("no")
    
    if i > 20:
        break

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [24]:
tree.label()[0].semantics()

<ApplicationExpression '@Is'('@Word'('daughter'),'@between'('@And'('ArgY','ArgX')))>

In [25]:
type(tree.label()[0].semantics())

nltk.sem.logic.ApplicationExpression

In [26]:
str(tree.label()[0].semantics())

"'@Is'('@Word'('daughter'),'@between'('@And'('ArgY','ArgX')))"

In [27]:
def parse_sem(sem):
    sem_split = re.split(',|(\()',sem)
    while None in sem_split:
        sem_split.remove(None)
    new_sem = []
    for i,w in enumerate(sem_split):
        if w=='(':
            new_sem.insert(-1,'(')
        else:
            new_sem.append(sem_split[i])
    sem_ = ''
    for i,s in enumerate(new_sem):
        sem_replace = s.replace('SPACE', ' ')
        for com in comma_index:
            sem_replace = sem_replace.replace('COMMA' + str(comma_index[com]), com)
        if sem_replace.startswith("'"):
            if sem_replace.endswith(")"):
                posi = len(sem_replace)-1
                while sem_replace[posi]==")":
                    posi-=1
                assert sem_replace[posi]=="\'"
            else:
                posi = len(sem_replace)-1
                assert sem_replace[posi] == "\'"

            sem_replace = sem_replace[0]+sem_replace[1:posi].replace('\'','\\\'')+sem_replace[posi:]
        if new_sem[i-1]=='(':
            sem_=sem_+sem_replace
        else:
            sem_ = sem_+','+sem_replace

    try:
        em = ('.root',eval(sem_[1:]))
        return em
    except:
        return False

comma_index = {',':0,'-LRB-':1,'-RRB-':2,'.':3,'-':4}

In [29]:
type(parse_sem(str(tree.label()[0].semantics())))

tuple

In [30]:
f = lambda x: lambda y: x*y

In [31]:
f(1)

<function __main__.<lambda>.<locals>.<lambda>(y)>

In [36]:
type(f(1))

function

In [37]:
f(1)(2)

2

In [38]:
temp = parse_sem(str(tree.label()[0].semantics()))

In [39]:
temp

('.root',
 ('@Is', ('@Word', 'daughter'), ('@between', ('@And', 'ArgY', 'ArgX'))))

In [43]:
ops[temp[0]]

<function __main__.<lambda>(xs)>

In [44]:
type(temp[0])

str

In [121]:
def recurse_soft(sem):
    try:
        if isinstance(sem, tuple):
            op = ops_soft[sem[0]]
            args = [recurse_soft(arg) for arg in sem[1:]]
            if False in args:
                return False
            return op(*args) if args else op
        else:
            if sem in syntax:
                return syntax[sem]
            else:
                return sem
    except:
        return False

In [49]:
# recurse(temp)("SUBJ's daughter appeared between OBJ .")

In [50]:
import numpy as np

In [52]:
np.exp(1)[0,0]

IndexError: invalid index to scalar variable.

In [95]:
hehe = np.zeros([2,1],np.float32)
hehe_1 = np.zeros([2,1],np.float32)

In [100]:
import numpy as np
np.dot(hehe.transpose(), hehe_1)[0][0]

0.0

In [2]:
nlp = StanfordCoreNLP("stanford-corenlp-4.1.0")

AccessDenied: psutil.AccessDenied (pid=8634)